# CHAPTER 7: Data Cleaning and Preparation

In [1]:
import pandas as pd
import numpy as np

## 7.1 Handling Missing Data

In [2]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

The `isna` method gives us a Boolean Series with True where values are null:

In [3]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

The built-in Python None value is also treated as NA:

In [ ]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])
string_data

,0
0,aardvark
1,NaN
2,None
3,avocado


In [ ]:
string_data.isna()

,0
0,False
1,True
2,True
3,False


In [ ]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data

,0
0,1.0
1,2.0
2,NaN


In [ ]:
float_data.isna()

,0
0,False
1,False
2,True


### Filtering Out Missing Data
There are a few ways to filter out missing data. While you always have the option to
do it by hand using `pandas.isna` and `Boolean` indexing, `dropna` can be helpful.

In [4]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

This is the same thing as doing:

In [5]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
[np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [7]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Passing `how="all"` will drop only rows that are all NA:

In [8]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


To drop columns in the same way, pass `axis="columns"`:

In [9]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [10]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Suppose you want to keep only rows containing at most a certain number of missing
observations. You can indicate this with the **thresh** argument:

In [11]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,-0.036834,NaN,NaN
1,-0.211305,NaN,NaN
2,0.944718,NaN,-0.449332
3,-2.056007,NaN,0.615580
4,0.484973,-1.030513,1.075664
5,0.532007,-1.331052,-0.435337
6,-1.211744,-0.575828,-1.720053


In [12]:
df.dropna()

,0,1,2
4,0.484973,-1.030513,1.075664
5,0.532007,-1.331052,-0.435337
6,-1.211744,-0.575828,-1.720053


In [13]:
df.dropna(thresh=2)

,0,1,2
2,0.944718,NaN,-0.449332
3,-2.056007,NaN,0.615580
4,0.484973,-1.030513,1.075664
5,0.532007,-1.331052,-0.435337
6,-1.211744,-0.575828,-1.720053


### Filling In Missing Data
For most
purposes, the `fillna` method is the workhorse function to use. Calling `fillna` with a
constant replaces missing values with that value:

In [14]:
df.fillna(0)

,0,1,2
0,-0.036834,0.000000,0.000000
1,-0.211305,0.000000,0.000000
2,0.944718,0.000000,-0.449332
3,-2.056007,0.000000,0.615580
4,0.484973,-1.030513,1.075664
5,0.532007,-1.331052,-0.435337
6,-1.211744,-0.575828,-1.720053


Calling `fillna` with a dictionary, you can use a different fill value for each column:

In [15]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,-0.036834,0.500000,0.000000
1,-0.211305,0.500000,0.000000
2,0.944718,0.500000,-0.449332
3,-2.056007,0.500000,0.615580
4,0.484973,-1.030513,1.075664
5,0.532007,-1.331052,-0.435337
6,-1.211744,-0.575828,-1.720053


The same interpolation methods available for reindexing can be used
with `fillna`:

In [16]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df

,0,1,2
0,-1.532870,-0.390837,-0.128167
1,0.304593,-0.367962,-0.264613
2,1.221243,NaN,-0.217228
3,0.354484,NaN,0.960626
4,0.629573,NaN,NaN
5,-0.343495,NaN,NaN


In [17]:
df.fillna(method="ffill")

C:\Users\Admin\AppData\Local\Temp\ipykernel_16684\3944122520.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill")


,0,1,2
0,-1.532870,-0.390837,-0.128167
1,0.304593,-0.367962,-0.264613
2,1.221243,-0.367962,-0.217228
3,0.354484,-0.367962,0.960626
4,0.629573,-0.367962,0.960626
5,-0.343495,-0.367962,0.960626


In [18]:
df.fillna(method="ffill", limit=2)

C:\Users\Admin\AppData\Local\Temp\ipykernel_16684\1627181726.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", limit=2)


,0,1,2
0,-1.532870,-0.390837,-0.128167
1,0.304593,-0.367962,-0.264613
2,1.221243,-0.367962,-0.217228
3,0.354484,-0.367962,0.960626
4,0.629573,NaN,0.960626
5,-0.343495,NaN,0.960626


With **fillna** you can do lots of other things such as simple data imputation using the
**median** or **mean** statistics:

In [19]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 Data Transformation
So far in this chapter we’ve been concerned with handling missing data. Filtering,
cleaning, and other transformations are another class of important operations.

### Removing Duplicates

In [20]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                    "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


The DataFrame method duplicated returns a Boolean Series indicating whether
each row is a duplicate or not.

In [ ]:
data.duplicated()

,0
0,False
1,False
2,False
3,False
4,False
5,False
6,True


Relatedly, `drop_duplicates` returns a DataFrame with rows where the duplicated
array is False filtered out:

In [ ]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


duplicates. Suppose we had an additional column of
values and wanted to filter duplicates based only on the "k1" column:

In [ ]:
data["v1"] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [ ]:
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


**duplicated** and **drop_duplicates** by default keep the first observed value combina‐
tion. Passing `keep="last"` will return the last one:

In [ ]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

In [ ]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
....: "pastrami", "corned beef", "bacon",
....: "pastrami", "honey ham", "nova lox"],
....: "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [ ]:
meat_to_animal = {
"bacon": "pig",
"pulled pork": "pig",
"pastrami": "cow",
"corned beef": "cow",
"honey ham": "pig",
"nova lox": "salmon"
}

The `map` method on a Series accepts a function or dictionary-like object containing a mapping to do
the transformation of values:

In [ ]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


We could also have passed a function that does all the work:


In [ ]:
def get_animal(x):
  return meat_to_animal[x]

data["food"].map(get_animal)

,food
0,pig
1,pig
2,pig
3,cow
4,cow
5,pig
6,cow
7,pig
8,salmon


Using `map` is a convenient way to perform element-wise transformations and other
data cleaning-related operations.

### Replacing Values

In [21]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA
values that pandas understands, we can use `replace`, producing a new Series:

In [22]:
data.replace(-999, np.nan)


0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

If you want to replace multiple values at once, you instead pass a list and then the
substitute value:

In [23]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

To use a different replacement for each value, pass a list of substitutes:

In [24]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The argument passed can also be a dictionary:

In [25]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### Renaming Axis Indexes

In [26]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
        index=["Ohio", "Colorado", "New York"],
        columns=["one", "two", "three", "four"])

Like a Series, the axis indexes have a map method:

In [27]:
def transform(x):
    return x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

You can assign to the index attribute, modifying the DataFrame in place:

In [28]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


If you want to create a transformed version of a dataset without modifying the
original, a useful method is `rename`:

In [29]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [30]:
data.rename(index={"OHIO": "INDIANA"}, columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### Discretization and Binning

In [31]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

Let’s divide these into bins of 18 to 25, 26 to 35, 36 to 60, and finally 61 and older. To
do so, you have to use `pandas.cut`:

In [32]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [33]:
age_categories.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [34]:
age_categories.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [35]:
age_categories.categories[0]

Interval(18, 25, closed='right')

In [36]:
pd.value_counts(age_categories)

C:\Users\Admin\AppData\Local\Temp\ipykernel_16684\3010498523.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(age_categories)


(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

Note that `pd.value_counts(categories)` are the bin counts for the result of
`pandas.cut`.

In the string representation of an interval, a parenthesis means that the side is *open*
(exclusive), while the square bracket means it is *closed* (inclusive). You can change
which side is closed by passing right=False:

In [37]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

You can override the default interval-based bin labeling by passing a list or array to
the labels option:

In [38]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [39]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

[(0.73, 0.97], (0.26, 0.49], (0.73, 0.97], (0.73, 0.97], (0.49, 0.73], ..., (0.26, 0.49], (0.49, 0.73], (0.49, 0.73], (0.73, 0.97], (0.02, 0.26]]
Length: 20
Categories (4, interval[float64, right]): [(0.02, 0.26] < (0.26, 0.49] < (0.49, 0.73] < (0.73, 0.97]]

The `precision=2` option limits the decimal precision to two digits.

Since `pandas.qcut` uses sample
quantiles instead, you will obtain roughly equally sized bins:

In [40]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles

[(0.013, 0.65], (-0.71, 0.013], (0.013, 0.65], (-0.71, 0.013], (0.65, 3.08], ..., (-3.84, -0.71], (-0.71, 0.013], (-3.84, -0.71], (-0.71, 0.013], (-0.71, 0.013]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.84, -0.71] < (-0.71, 0.013] < (0.013, 0.65] < (0.65, 3.08]]

In [41]:
pd.value_counts(quartiles)

C:\Users\Admin\AppData\Local\Temp\ipykernel_16684\3472704981.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(quartiles)


(-3.84, -0.71]    250
(-0.71, 0.013]    250
(0.013, 0.65]     250
(0.65, 3.08]      250
Name: count, dtype: int64

Similar to `pandas.cut`, you can pass your own quantiles (numbers between 0 and 1,
inclusive):

In [42]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.827, -1.281]    100
(-1.281, 0.0132]    400
(0.0132, 1.313]     400
(1.313, 3.076]      100
Name: count, dtype: int64

### Detecting and Filtering Outliers

In [43]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.043718,0.013192,0.013347,0.060305
std,1.013789,1.035862,0.977446,0.979399
min,-3.803700,-3.191103,-2.717773,-3.387956
25%,-0.665616,-0.678308,-0.622705,-0.623589
50%,0.042164,0.026828,0.054024,0.057052
75%,0.698456,0.705646,0.668217,0.754741
max,3.733027,3.448523,3.166702,3.291504


Suppose you wanted to find values in one of the columns exceeding 3 in absolute
value:

In [44]:
col = data[2]
col[col.abs() > 3]

220    3.092846
841    3.166702
Name: 2, dtype: float64

To select all rows having a value exceeding 3 or –3, you can use the any method on a
Boolean DataFrame:

In [45]:
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
65,3.733027,0.373311,1.048945,0.832121
210,-0.550778,-3.191103,0.186680,-0.894177
220,-1.989444,-0.073138,3.092846,0.088253
439,3.130158,-0.398022,0.221630,0.202236
579,3.597703,-0.703554,0.682117,-0.644546
650,-0.872874,3.448523,0.153359,-0.160992
656,0.041761,-0.814087,0.094215,3.291504
769,-0.110080,0.730125,-1.273941,-3.387956
841,0.730430,-1.138075,3.166702,0.359607
850,-3.803700,1.295731,-0.779413,0.278100


Values can be set based on these criteria. Here is code to cap values outside the
interval –3 to 3:

In [46]:
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.043061,0.013087,0.013088,0.060401
std,1.006313,1.033438,0.976633,0.977186
min,-3.000000,-3.000000,-2.717773,-3.000000
25%,-0.665616,-0.678308,-0.622705,-0.623589
50%,0.042164,0.026828,0.054024,0.057052
75%,0.698456,0.705646,0.668217,0.754741
max,3.000000,3.000000,3.000000,3.000000


The statement` np.sign(data)` produces 1 and –1 values based on whether the values
in data are positive or negative:

In [47]:
np.sign(data).head()

,0,1,2,3
0,1.0,1.0,1.0,-1.0
1,1.0,1.0,1.0,1.0
2,1.0,-1.0,1.0,-1.0
3,-1.0,1.0,-1.0,-1.0
4,1.0,-1.0,-1.0,1.0


### Permutation and Random Sampling
Permuting (randomly reordering) a Series or the rows in a DataFrame is possible
using the numpy.random.permutation function. Calling permutation with the length
of the axis you want to permute produces an array of integers indicating the new
ordering:

In [48]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34


In [49]:
sampler = np.random.permutation(5)
sampler

array([3, 1, 0, 4, 2])

That array can then be used in iloc-based indexing or the equivalent take function:

In [50]:
df.take(sampler)

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20


In [51]:
df.iloc[sampler]

,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
1,7,8,9,10,11,12,13
0,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
2,14,15,16,17,18,19,20


By invoking take with `axis="columns"`, we could also select a permutation of the
columns:

In [52]:
column_sampler = np.random.permutation(7)
column_sampler


array([0, 5, 6, 1, 2, 3, 4])

In [53]:
df.take(column_sampler, axis="columns")

,0,5,6,1,2,3,4
0,0,5,6,1,2,3,4
1,7,12,13,8,9,10,11
2,14,19,20,15,16,17,18
3,21,26,27,22,23,24,25
4,28,33,34,29,30,31,32


To select a random subset without replacement (the same row cannot appear twice),
you can use the `sample` method on Series and DataFrame:

In [54]:
df.sample(n=3)

,0,1,2,3,4,5,6
2,14,15,16,17,18,19,20
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27


To generate a sample *with* replacement (to allow repeat choices), pass `replace=True`
to sample:

In [55]:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

4    4
0    5
3    6
1    7
2   -1
3    6
2   -1
4    4
2   -1
2   -1
dtype: int64

### Computing Indicator/Dummy Variables

In [56]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
    "data1": range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [57]:
pd.get_dummies(df["key"])

,a,b,c
0,False,True,False
1,False,True,False
2,True,False,False
3,False,False,True
4,True,False,False
5,False,True,False


In some cases, you may want to add a prefix to the columns in the indicator Data‐
Frame, which can then be merged with the other data. `pandas.get_dummies` has a
prefix argument for doing this:

In [58]:
dummies = pd.get_dummies(df["key"], prefix="key")
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,False,True,False
1,1,False,True,False
2,2,True,False,False
3,3,False,False,True
4,4,True,False,False
5,5,False,True,False


The `DataFrame.join` method will be explained in more detail in the next chapter.

If a row in a DataFrame belongs to multiple categories, we have to use a different
approach to create the dummy variables. Let’s look at the MovieLens 1M dataset.

In [ ]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("datasets/movielens/movies.dat", sep="::",
header=None, names=mnames, engine="python")
movies[:10]

pandas has implemented a special Series method `str.get_dummies` that handles this scenario of multiple group membership encoded
as a delimited string:

In [ ]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

Then, as before, you can combine this with movies while adding a `"Genre_"` to the
column names in the `dummies` DataFrame with the `add_prefix` method:

In [ ]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.iloc[0]

A useful recipe for statistical applications is to combine  `pandas.get_dummies` with a
discretization function like `pandas.cut`:

In [60]:
np.random.seed(12345) # to make the example repeatable
values = np.random.uniform(size=10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [61]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


## 7.3 Extension Data Types

Let’s look at an example where we create a Series of integers with a missing value:

In [62]:
s = pd.Series([1, 2, 3, None])
s

0    1.0
1    2.0
2    3.0
3    NaN
dtype: float64

In [63]:
s.dtype

dtype('float64')

Mainly for backward compatibility reasons, Series uses the legacy behavior of using
a float64 data type and np.nan for the missing value. We could create this Series
instead using pandas.Int64Dtype:

In [64]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s

0       1
1       2
2       3
3    <NA>
dtype: Int64

In [65]:
s.isna()

0    False
1    False
2    False
3     True
dtype: bool

In [66]:
s.dtype

Int64Dtype()

The output `<NA>` indicates that a value is missing for an extension type array. This
uses the special `pandas.NA` sentinel value:

In [67]:
print(s[3])
print(s[3] is pd.NA)

<NA>
True


We also could have used the shorthand "Int64" instead of pd.Int64Dtype() to
specify the type.

In [68]:
s = pd.Series([1, 2, 3, None], dtype="Int64")

pandas also has an extension type specialized for string data that does not use
NumPy object arrays

In [69]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

0      one
1      two
2     <NA>
3    three
dtype: string

Extension types can be passed to the Series `astype` method, allowing you to convert
easily as part of your data cleaning process:

In [70]:
df = pd.DataFrame({"A": [1, 2, None, 4], 
                   "B": ["one", "two", "three", None], 
                   "C": [False, None, False, True]})
df

,A,B,C
0,1.0,one,False
1,2.0,two,None
2,NaN,three,False
3,4.0,None,True


In [71]:
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


## 7.4 String Manipulation
Python has long been a popular raw data manipulation language in part due to its
ease of use for string and text processing. Most text operations are made simple
with the string object’s built-in methods. For more complex pattern matching and
text manipulations, regular expressions may be needed. pandas adds to the mix by
enabling you to apply string and regular expressions concisely on whole arrays of
data, additionally handling the annoyance of missing data.

### Python Built-In String Object Methods

In [72]:
val = "a,b, guido"
val.split(",")

['a', 'b', ' guido']

`split` is often combined with strip to trim whitespace (including line breaks):

In [73]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

These substrings could be concatenated together with a two-colon delimiter using
addition:

In [74]:
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

But this isn’t a practical generic method. A faster and more Pythonic way is to pass a
list or tuple to the join method on the string "::":

In [75]:
"::".join(pieces)

'a::b::guido'

Using Python’s in keyword is
the best way to detect a substring, though index and find can also be used:

In [76]:
print("guido" in val)
print(val.index(","))
print(val.find(":"))

True
1
-1


Note that the difference between find and index is that index raises an exception if
the string isn’t found (versus returning –1):

In [77]:
val.index(":")

ValueError: substring not found

Relatedly, count returns the number of occurrences of a particular substring:

In [78]:
val.count(",")

2

`replace` will substitute occurrences of one pattern for another. It is commonly used
to delete patterns, too, by passing an empty string:

In [79]:
val.replace(",", "::")

'a::b:: guido'

In [80]:
val.replace(",", "")

'ab guido'

### Regular Expressions

The regex describing one or more whitespace characters is \s+:

In [2]:
import re
text = "foo bar\t baz \tqux"
re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

When you call `re.split(r"\s+", text)`, the regular expression is first *compiled*, and
then its split method is called on the passed text. You can compile the regex yourself
with `re.compile`, forming a reusable regex object:

In [3]:
regex = re.compile(r"\s+")
regex.split(text)

['foo', 'bar', 'baz', 'qux']

If, instead, you wanted to get a list of all patterns matching the regex, you can use the
`findall` method:

In [4]:
regex.findall(text)

[' ', '\t ', ' \t']

`match` and `search` are closely related to findall. While `findall` returns all matches
in a string, `search` returns only the first match. More rigidly, `match` *only* matches at
the beginning of the string. As a less trivial example, let’s consider a block of text and
a regular expression capable of identifying most email addresses:

In [5]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

In [6]:
regex = re.compile(pattern, flags=re.IGNORECASE)

Using `findall` on the text produces a list of the email addresses:

In [7]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search` returns a special match object for the first email address in the text. For the
preceding regex, the match object can only tell us the start and end position of the
pattern in the string:

In [8]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [9]:
text[m.start():m.end()]

'dave@google.com'

`regex.match` returns **None**, as it will match only if the pattern occurs at the start of the
string:

In [10]:
print(regex.match(text))

None


Relatedly, `sub` will return a new string with occurrences of the pattern replaced by a
new string:

In [11]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED


Suppose you wanted to find email addresses and simultaneously segment each
address into its three components: username, domain name, and domain suffix. To
do this, put parentheses around the parts of the pattern to segment:

In [12]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

A match object produced by this modified regex returns a tuple of the pattern
components with its groups method:

In [13]:
m = regex.match("wesm@bright.net")
m.groups()

('wesm', 'bright', 'net')

`findall` returns a list of tuples when the pattern has groups:

In [15]:
regex.findall(text)


[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

`sub` also has access to groups in each match using special symbols like \1 and \2. The
symbol \1 corresponds to the first matched group, \2 corresponds to the second, and
so forth:

In [16]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


### String Functions in pandas
Cleaning up a messy dataset for analysis often requires a lot of string manipulation.
To complicate matters, a column containing strings will sometimes have missing data:

In [18]:
import pandas as pd
import numpy as np

In [19]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [20]:
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

String and regular expression methods can be applied (passing a lambda or other
function) to each value using `data.map`, but it will fail on the NA (null) values.

In [21]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

Note that the result of this operation has an **object dtype**. pandas has *extension types*
that provide for specialized treatment of strings, integers, and Boolean data which
until recently have had some rough edges when working with missing data:

In [22]:
data_as_string_ext = data.astype('string')
data_as_string_ext

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                 <NA>
dtype: string

In [23]:
data_as_string_ext.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

Regular expressions can be used, too, along with any re options like `IGNORECASE`:

In [25]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

There are a couple of ways to do vectorized element retrieval. Either use `str.get` or
`index into` the str attribute:

In [26]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches

Dave     (dave, google, com)
Steve    (steve, gmail, com)
Rob        (rob, gmail, com)
Wes                      NaN
dtype: object

In [27]:
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

You can similarly slice strings using this syntax:

In [28]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

The `str.extract` method will return the captured groups of a regular expression as a
DataFrame:

In [29]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


## 7.5 Categorical Data

### Background and Motivation
Frequently, a column in a table may contain repeated instances of a smaller set of
distinct values. We have already seen functions like unique and `value_counts`, which
enable us to extract the distinct values from an array and compute their frequencies,
respectively:

In [30]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [31]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [32]:
pd.value_counts(values)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2152\3297668723.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(values)


apple     6
orange    2
Name: count, dtype: int64

In data warehousing, a best practice is to use
so-called *dimension tables* containing the distinct values and storing the primary
observations as integer keys referencing the dimension table:

In [33]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [34]:
dim

0     apple
1    orange
dtype: object

### Categorical Extension Type in pandas
pandas has a special Categorical extension type for holding data that uses the
integer-based categorical representation or *encoding*. This is a popular data compres‐
sion technique for data with many occurrences of similar values and can provide
significantly faster performance with lower memory use, especially for string data.

Let’s consider the example Series from before:

In [35]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({'fruit': fruits,
                    'basket_id': np.arange(N),
                    'count': rng.integers(3, 15, size=N),
                    'weight': rng.uniform(0, 4, size=N)},
                    columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


Here, `df['fruit']` is an array of Python string objects. We can convert it to categori‐
cal by calling:

In [36]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

The values for `fruit_cat` are now an instance of `pandas.Categorical`, which you
can access via the `.array` attribute:

In [37]:
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

The `Categorical` object has categories and codes attributes:

In [38]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [39]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

A useful trick to get a mapping between codes and categories is:

In [40]:
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

You can convert a DataFrame column to categorical by assigning the converted result:

In [41]:
df['fruit'] = df['fruit'].astype('category')
df["fruit"]

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

You can also create `pandas.Categorical` directly from other types of Python
sequences:

In [42]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

If you have obtained categorical encoded data from another source, you can use the
alternative `from_codes` constructor:

In [43]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [44]:
ordered_cat = pd.Categorical.from_codes(codes, categories, ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

The output [foo < bar < baz] indicates that 'foo' precedes 'bar' in the ordering,
and so on. An unordered categorical instance can be made ordered with `as_ordered`:

In [45]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### Computations with Categoricals
Using `Categorical` in pandas compared with the nonencoded version (like an array
of strings) generally behaves the same way. Some parts of pandas, like the `groupby`
function, perform better when working with categoricals. There are also some func‐
tions that can utilize the `ordered` flag.

In [46]:
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

Let’s compute a quartile binning of this data and extract some statistics:

In [47]:
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

While useful, the exact sample quartiles may be less useful for producing a report
than quartile names. We can achieve this with the `labels` argument to qcut:

In [48]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [49]:
bins.codes[:10]

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

The labeled bins categorical does not contain information about the bin edges in the
data, so we can use groupby to extract some summary statistics:

In [50]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
            .groupby(bins)
            .agg(['count', 'min', 'max'])
            .reset_index())
results

C:\Users\Admin\AppData\Local\Temp\ipykernel_2152\420364718.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(bins)


,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


The `'quartile'` column in the result retains the original categorical information,
including ordering, from `bins`:

In [51]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

**Better performance with categoricals**

In [52]:
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

Now we convert `labels` to categorical:

In [53]:
categories = labels.astype('category')

Now we note that `labels` uses significantly more memory than `categories`:

In [54]:
labels.memory_usage(deep=True)

520000132

In [55]:
categories.memory_usage(deep=True)

10000512

The conversion to category is not free, of course, but it is a one-time cost:

In [56]:
%time _ = labels.astype('category')

CPU times: total: 281 ms
Wall time: 295 ms


GroupBy operations can be significantly faster with categoricals because the underly‐
ing algorithms use the integer-based codes array instead of an array of strings. Here
we compare the performance of `value_counts()`, which internally uses the GroupBy
machinery:

In [57]:
%timeit labels.value_counts()

363 ms ± 29.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%timeit categories.value_counts()

42.9 ms ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Categorical Methods
Series containing categorical data have several special methods similar to the `Ser
ies.str` specialized string methods. This also provides convenient access to the
categories and codes.

In [59]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

The special *accessor* attribute `cat` provides access to categorical methods:

In [60]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

Suppose that we know the actual set of categories for this data extends beyond the
four values observed in the data. We can use the `set_categories` method to change
them:

In [61]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

While it appears that the data is unchanged, the new categories will be reflected
in operations that use them. For example, `value_counts` respects the categories, if
present:

In [62]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
Name: count, dtype: int64

In [63]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

After you filter a large DataFrame or Series, many
of the categories may not appear in the data. To help with this, we can use the
`remove_unused_categories` method to trim unobserved categories:

In [64]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [65]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

**Creating dummy variables for modeling**
When you’re using statistics or machine learning tools, you’ll often transform catego‐
rical data into *dummy variables*, also known as *one-hot* encoding. This involves creat‐
ing a DataFrame with a column for each distinct category; these columns contain 1s
for occurrences of a given category and 0 otherwise.

In [66]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

As mentioned previously in this chapter, the `pandas.get_dummies` function converts
this one-dimensional categorical data into a DataFrame containing the dummy
variable:

In [67]:
pd.get_dummies(cat_s)

,a,b,c,d
0,True,False,False,False
1,False,True,False,False
2,False,False,True,False
3,False,False,False,True
4,True,False,False,False
5,False,True,False,False
6,False,False,True,False
7,False,False,False,True


## 7.6 Conclusion